In [90]:
import random
import gurobipy as gp
from gurobipy import GRB
import pandas as pd
import numpy as np
import math

#可直接在K and N_F新增起火點和消防員的index
#設定Q b H為隨機產生或是給定
mode = "random"
filename1="fighter_route.xlsx"
filename2="fire_route.xlsx"
N_number=30
N = set([i for i in range(1,N_number+1)])  #所有點的集合
# T_number =200 
T_number = 200
T = list([i for i in range(T_number+1)])    #時間集合
K = set([1])                        #K=消防車集合
N_D = set([18])                      #Depot
N_F = set([5])                      #起火點
H={}
Q={}
b={}
#rand_fightertraveltime=[]
#rand_firetraveltime=[]
# H={1:4,2:3,3:6,4:1,5:2,6:2,8:1,9:6,10:5}
# Q={1:3,2:4,3:5,4:3,5:3,6:4,8:1,9:7,10:10}
# b={1:1,2:2,3:3,4:4,5:5,6:6,8:8,9:9,10:10}
P={1:3,2:4}                                  #各個消防車單位時間處理的燃料量

if mode == "random":
    for i in range(1,N_number+1):
        if i not in N_D:
            #Q[i] = random.randint(5,15)
            Q[i]=5
            #b[i] = random.randint(50,100)
            b[i]=30
            #H[i] = random.randint(5,15)
            H[i] = 5
else:
    H={1:1,2:2,3:1,4:2,5:4,6:3,7:3,8:3,9:2,10:2,11:2,12:4,13:2,15:3,16:6,17:5,18:2,19:1,20:4}                  #q=單位時間火在i點燒的燃料量
    Q={1:2,2:3,3:4,4:4,5:3,6:2,7:6,8:5,9:4,10:6,11:9,12:8,13:3,15:5,16:3,17:3,18:4,19:6,20:7}                  #h=火燒i點需要的時間
    b={1:40,2:40,3:30,4:50,5:70,6:30,7:70,8:40,9:40,10:50,11:60,12:50,13:40,15:60,16:20,17:60,18:20,19:40,20:80}#b=節點i的價值
       

M=10
A_p=gp.tuplelist()
A_f=gp.tuplelist()
tau=gp.tupledict()
lamb=gp.tupledict()
process={}
for k in K:
    process[k]={}
    for i in Q:
        process[k][i] = math.ceil(Q[i]*H[i]/P[k])


print(process)       
#print("fighter:",rand_fightertraveltime)
#print("fire:",rand_firetraveltime)
print("Q:",Q)
print("H:",H)
print("b:",b)



{1: {1: 9, 2: 9, 3: 9, 4: 9, 5: 9, 6: 9, 7: 9, 8: 9, 9: 9, 10: 9, 11: 9, 12: 9, 13: 9, 14: 9, 15: 9, 16: 9, 17: 9, 19: 9, 20: 9, 21: 9, 22: 9, 23: 9, 24: 9, 25: 9, 26: 9, 27: 9, 28: 9, 29: 9, 30: 9}}
Q: {1: 5, 2: 5, 3: 5, 4: 5, 5: 5, 6: 5, 7: 5, 8: 5, 9: 5, 10: 5, 11: 5, 12: 5, 13: 5, 14: 5, 15: 5, 16: 5, 17: 5, 19: 5, 20: 5, 21: 5, 22: 5, 23: 5, 24: 5, 25: 5, 26: 5, 27: 5, 28: 5, 29: 5, 30: 5}
H: {1: 5, 2: 5, 3: 5, 4: 5, 5: 5, 6: 5, 7: 5, 8: 5, 9: 5, 10: 5, 11: 5, 12: 5, 13: 5, 14: 5, 15: 5, 16: 5, 17: 5, 19: 5, 20: 5, 21: 5, 22: 5, 23: 5, 24: 5, 25: 5, 26: 5, 27: 5, 28: 5, 29: 5, 30: 5}
b: {1: 30, 2: 30, 3: 30, 4: 30, 5: 30, 6: 30, 7: 30, 8: 30, 9: 30, 10: 30, 11: 30, 12: 30, 13: 30, 14: 30, 15: 30, 16: 30, 17: 30, 19: 30, 20: 30, 21: 30, 22: 30, 23: 30, 24: 30, 25: 30, 26: 30, 27: 30, 28: 30, 29: 30, 30: 30}


In [91]:
df = pd.read_excel(filename1)
df_num = len(df.index)
for i in N:
    A_p.append((i,i))
for i in range(df_num):
    u = int(df.iloc[i]['i'])
    v = int(df.iloc[i]['j'])
    fighterIndex = df.iloc[i]['k']
    time = df.iloc[i]['travel time']
    tau[u,v,fighterIndex] = time
    #避免在多消防員時重複紀錄arc set
    if (u,v) not in A_p:
        A_p.append((u,v))
       
       
    
df = pd.read_excel(filename2)
df_num = len(df.index)
for i in range(df_num):
    u = int(df.iloc[i]['i'])
    v = int(df.iloc[i]['j'])
    time = df.iloc[i]['travel time']
    lamb[u,v] = time
    A_f.append((u,v))
#print(A_p)
#print(A_f)
# print(tau)
# print(lamb)

A_f_NEIGHBOR={}                      #A_f_NEIGHBOR=與點i相鄰的點
    
for l in N-N_D:                          #定義A_f_NEIGHBOR
    connect = A_f.select('*',l)
    A_f_NEIGHBOR[l]=[]
    for temp in connect:
        A_f_NEIGHBOR[l].append(temp[0])
                      
A_f_NEIGHBOR_T={}                    #A_f_NEIGHBOR_T=紀錄 t-hi-Lambda(i,j)>=0 且 與j點相鄰的i點

for j in N-N_D-N_F:                         #定義A_f_NEIGHBOR_T
    for t in T:
        A_f_NEIGHBOR_T[j,t]=[]
        for i in A_f_NEIGHBOR[j]:
            if t-H[i]-lamb[i,j]>=0:
                A_f_NEIGHBOR_T[j,t].append(i)
model=gp.Model("FIREFIGHTER")


x={}
w={}
u={}
u_bar={}
v={}
v_bar={}
epsilon=1e-3

#定義x[i,j,k,t]
for k in K:
    for t in T:
        for i in range(len(A_p)):
                x[A_p[i][0],A_p[i][1],k,t]=model.addVar(vtype='B',name="x[%d,%d,%d,%d]" %(A_p[i][0],A_p[i][1],k,t))

#定義w[i,k,t]
for k in K:
    for t in T:
        for i in N:
            w[i,k,t]=model.addVar(vtype='B',name="w[%d,%d,%d]" %(i,k,t))

#定義u[i,t]
u = model.addVars(N, T, vtype="B",name="u")

#定義u_bar[i,k,t]
u_bar = model.addVars(N, K, T, vtype="B", name="u_bar")

#定義v[i,t]
v = model.addVars(N, T, vtype="B", name="v")
        
#定義v_bar[i,t]
v_bar = model.addVars(N, T, vtype="B", name="v_bar")        
        
model.update()

for k in K:                        #原點flow blance
    model.addConstr(gp.quicksum(x[i,j,k,0] for i,j in A_p) <= 1)
    
#depart from depot
for O in N_D:
    connect = A_p.select(O,'*')
    for k in K:
        model.addConstr(gp.quicksum(x[i,j,k,0] for i, j in connect) == 1)
#depot can't be processed
model.addConstrs((u_bar[i,k,t]==0 for i in N_D for k in K for t in range(T_number+1)))

#flow balance
for k in K:
    for t in range(1,T_number):
        for j in N: 
            in_connect = A_p.select('*',j)
            out_connect = A_p.select(j,'*')
            temp=0 #in-degree
            temp += w[j,k,t-1] # t-1在j idle
            if j in N_D: #j in depot set會有u_bar，只是都為0
                temp+=u_bar[j,k,t]
            else: #j not in depot set若現在的t>process time，則會有u_bar
                if process[k][j] <= t:
                    temp += u_bar[j,k,t-process[k][j]]
            for m,n in in_connect:
                if m != n and tau[m,n,k] <= t: #若現在的t>travel time，則會有x
                    temp += x[m,n,k,t-tau[m,n,k]]
            model.addConstr(temp == gp.quicksum(x[n,w,k,t] for n,w in out_connect),name="flow") #in-degree = out-degree
model.addConstrs((u_bar[i,k,t] + w[i,k,t] == x[i,i,k,t] for i in N-N_D for k in K for t in range(T_number)))
model.addConstrs((w[s,k,t] == x[s,s,k,t]) for s in N_D for k in K for t in range(T_number))

#######################################################################################################################
model.addConstrs(u[i, t]+u_bar.sum(i, '*', t) <= 1 for i in N for t in T) #constrain 6
model.addConstrs(v[i, t] + u[i, t] == v[i, t+1] for i in N for t in T[0:-1]) #constrain 7
model.addConstrs(v_bar[i, t] + u_bar.sum(i, '*', t) == v_bar[i, t+1] for i in N for t in T[0:-1]) #constrain 9
model.addConstrs(u_bar.sum(s, '*', '*') == 0 for s in N_D) #constrain 11
model.addConstrs(u.sum(s, '*') == 0 for s in N_D) #constrain 12
        
#######################################################################################################################
    
for j in N-N_D-N_F:                            #火焰的延燒
    for t in range(T_number):
        if len(A_f_NEIGHBOR_T[j,t])==0:
            model.addConstr(u[j,t] == 0,name='test')
        else:
            model.addConstr(gp.quicksum(u[i,t-H[i]-lamb[i,j]] for i in A_f_NEIGHBOR_T[j,t]) / M <= u[j,t] + v[j,t] + v_bar[j,t+1])
            model.addConstr(gp.quicksum(u[i,t-H[i]-lamb[i,j]] for i in A_f_NEIGHBOR_T[j,t]) >= u[j,t])
            
for i in N_F:                           #起火點起火
    model.addConstr(u[i,0]==1)
    
for i in N:                            #所有節點不會在t-1~t已經被燒
    model.addConstr(v[i,0]==0)
    
for i in N:                            #所有節點不會在t-1~t已經被澆水
    model.addConstr(v_bar[i,0]==0)
            
for k in K:                             #消防員不能去已經被燃燒的節點
    for t in T:
        for l in range(len(A_p)):
            if A_p[l][1] not in N_D:
                if A_p[l][0] ==  A_p[l][1]:
                    if t+2 <= T_number:
                        model.addConstr(M*(1-v[A_p[l][1],t+1]) >= x[A_p[l][0], A_p[l][1],k,t])
                    else:
                        model.addConstr(M*(1-v[A_p[l][1],T_number]) >= x[A_p[l][0], A_p[l][1],k,t])
                elif t + tau[A_p[l][0],A_p[l][1],k] + 1 <= T_number:
                    model.addConstr(M*(1-v[A_p[l][1],t + tau[A_p[l][0],A_p[l][1],k]]) >= x[A_p[l][0],A_p[l][1],k,t])
                else:
                    model.addConstr(M*(1-v[A_p[l][1],T_number]) >= x[A_p[l][0],A_p[l][1],k,t])
                    
                    
# model.setObjective(gp.quicksum(gp.quicksum(u[i,t] for t in T) * b[i] for i in N-N_D) + 
#                    gp.quicksum(epsilon*x[i,j,k,t] for (i,j,k,t) in x if i!=j),GRB.MINIMIZE)
model.setObjective(gp.quicksum(gp.quicksum(u[i,t] for t in T) * b[i] for i in N-N_D) + 
                   gp.quicksum(epsilon*x[i,j,k,t] for (i,j,k,t) in x if i!=j)+
                   gp.quicksum(epsilon*t*u_bar[i,k,t] for i in N-N_D for k in K for t in T),GRB.MINIMIZE)

model.optimize()
model.write('test.lp')
print("optimal value : ",model.ObjVal)
                    

Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 74227 rows, 64320 columns and 261643 nonzeros
Model fingerprint: 0x74087177
Variable types: 0 continuous, 64320 integer (64320 binary)
Coefficient statistics:
  Matrix range     [1e-01, 1e+01]
  Objective range  [1e-03, 3e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+01]
Presolve removed 49758 rows and 41013 columns
Presolve time: 3.52s
Presolved: 24469 rows, 23307 columns, 92990 nonzeros
Variable types: 0 continuous, 23307 integer (23307 binary)
Found heuristic solution: objective 840.0410000

Root relaxation: objective 5.074012e+02, 7206 iterations, 0.93 seconds (1.15 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0  507.40115    0 1705  840.04100  507.40115  39.6%     -    6s
H    0  

In [92]:
EPS=1.e-2
print("x:")
step=1
for k in K:
    print()
    print("消防員%d的路徑" % k)
    temp = [elem for elem in x if elem[2]==k]
    for (i,j,k,t) in temp:
        if x[i,j,k,t].X >EPS:
            #print("x[%d,%d,%d,%d]" %(i,j,k,t) , x[i,j,k,t].X)
            if i != j:
                print("在時刻 %d 從node%d 移動到 node%d" %(t,i,j)," ,travel time:",tau[i,j,k])
            else:            
                if u_bar[i,k,t].X == 1:
                    print("在時刻 %d 對node%d進行保護"%(t,i)," ,processing time:",process[k][i])
                else:
                    print("在時刻 %d 在node%d idle" %(t,i))

print("w:")
for (i,k,t) in w:
    if w[i,k,t].X>EPS:
        print("w[%d,%d,%d]" %(i,k,t) , w[i,k,t].X)

print("u:")
for (i,t) in u:
    if u[i,t].X>EPS:
        print("u[%d,%d]" %(i,t) , u[i,t].X)

print("u_bar:")
for (i,k,t) in u_bar:
    if u_bar[i,k,t].X>EPS:
        print("u_bar[%d,%d,%d]" %(i,k,t) , u_bar[i,k,t].X)

print("v:")
for (i,t) in v:
    if v[i,t].X>EPS:
        print("v[%d,%d]" %(i,t) , v[i,t].X)

print("v_bar:")
for (i,t) in v_bar:
    if v_bar[i,t].X>EPS:
        print("v_bar[%d,%d]" %(i,t) , v_bar[i,t].X) 

x:

消防員1的路徑
在時刻 0 從node18 移動到 node24  ,travel time: 10.0
在時刻 10 對node24進行保護  ,processing time: 9
在時刻 19 從node24 移動到 node15  ,travel time: 30.0
在時刻 49 對node15進行保護  ,processing time: 9
在時刻 58 從node15 移動到 node3  ,travel time: 23.0
在時刻 81 對node3進行保護  ,processing time: 9
在時刻 90 在node3 idle
在時刻 91 在node3 idle
在時刻 92 在node3 idle
在時刻 93 在node3 idle
在時刻 94 在node3 idle
在時刻 95 在node3 idle
在時刻 96 在node3 idle
在時刻 97 在node3 idle
在時刻 98 在node3 idle
在時刻 99 在node3 idle
在時刻 100 在node3 idle
在時刻 101 在node3 idle
在時刻 102 在node3 idle
在時刻 103 在node3 idle
在時刻 104 在node3 idle
在時刻 105 在node3 idle
在時刻 106 在node3 idle
在時刻 107 在node3 idle
在時刻 108 在node3 idle
在時刻 109 在node3 idle
在時刻 110 在node3 idle
在時刻 111 在node3 idle
在時刻 112 在node3 idle
在時刻 113 在node3 idle
在時刻 114 在node3 idle
在時刻 115 在node3 idle
在時刻 116 在node3 idle
在時刻 117 在node3 idle
在時刻 118 在node3 idle
在時刻 119 在node3 idle
在時刻 120 在node3 idle
在時刻 121 在node3 idle
在時刻 122 在node3 idle
在時刻 123 在node3 idle
在時刻 124 在node3 idle
在時刻 125 在node3 idle
在時刻 126 在node3 idle
在時刻

v[11,183] 1.0
v[11,184] 1.0
v[11,185] 1.0
v[11,186] 1.0
v[11,187] 1.0
v[11,188] 1.0
v[11,189] 1.0
v[11,190] 1.0
v[11,191] 1.0
v[11,192] 1.0
v[11,193] 1.0
v[11,194] 1.0
v[11,195] 1.0
v[11,196] 1.0
v[11,197] 1.0
v[11,198] 1.0
v[11,199] 1.0
v[11,200] 1.0
v[13,75] 1.0
v[13,76] 1.0
v[13,77] 1.0
v[13,78] 1.0
v[13,79] 1.0
v[13,80] 1.0
v[13,81] 1.0
v[13,82] 1.0
v[13,83] 1.0
v[13,84] 1.0
v[13,85] 1.0
v[13,86] 1.0
v[13,87] 1.0
v[13,88] 1.0
v[13,89] 1.0
v[13,90] 1.0
v[13,91] 1.0
v[13,92] 1.0
v[13,93] 1.0
v[13,94] 1.0
v[13,95] 1.0
v[13,96] 1.0
v[13,97] 1.0
v[13,98] 1.0
v[13,99] 1.0
v[13,100] 1.0
v[13,101] 1.0
v[13,102] 1.0
v[13,103] 1.0
v[13,104] 1.0
v[13,105] 1.0
v[13,106] 1.0
v[13,107] 1.0
v[13,108] 1.0
v[13,109] 1.0
v[13,110] 1.0
v[13,111] 1.0
v[13,112] 1.0
v[13,113] 1.0
v[13,114] 1.0
v[13,115] 1.0
v[13,116] 1.0
v[13,117] 1.0
v[13,118] 1.0
v[13,119] 1.0
v[13,120] 1.0
v[13,121] 1.0
v[13,122] 1.0
v[13,123] 1.0
v[13,124] 1.0
v[13,125] 1.0
v[13,126] 1.0
v[13,127] 1.0
v[13,128] 1.0
v[13,129] 1.0
v[1

v[27,185] 1.0
v[27,186] 1.0
v[27,187] 1.0
v[27,188] 1.0
v[27,189] 1.0
v[27,190] 1.0
v[27,191] 1.0
v[27,192] 1.0
v[27,193] 1.0
v[27,194] 1.0
v[27,195] 1.0
v[27,196] 1.0
v[27,197] 1.0
v[27,198] 1.0
v[27,199] 1.0
v[27,200] 1.0
v[28,56] 1.0
v[28,57] 1.0
v[28,58] 1.0
v[28,59] 1.0
v[28,60] 1.0
v[28,61] 1.0
v[28,62] 1.0
v[28,63] 1.0
v[28,64] 1.0
v[28,65] 1.0
v[28,66] 1.0
v[28,67] 1.0
v[28,68] 1.0
v[28,69] 1.0
v[28,70] 1.0
v[28,71] 1.0
v[28,72] 1.0
v[28,73] 1.0
v[28,74] 1.0
v[28,75] 1.0
v[28,76] 1.0
v[28,77] 1.0
v[28,78] 1.0
v[28,79] 1.0
v[28,80] 1.0
v[28,81] 1.0
v[28,82] 1.0
v[28,83] 1.0
v[28,84] 1.0
v[28,85] 1.0
v[28,86] 1.0
v[28,87] 1.0
v[28,88] 1.0
v[28,89] 1.0
v[28,90] 1.0
v[28,91] 1.0
v[28,92] 1.0
v[28,93] 1.0
v[28,94] 1.0
v[28,95] 1.0
v[28,96] 1.0
v[28,97] 1.0
v[28,98] 1.0
v[28,99] 1.0
v[28,100] 1.0
v[28,101] 1.0
v[28,102] 1.0
v[28,103] 1.0
v[28,104] 1.0
v[28,105] 1.0
v[28,106] 1.0
v[28,107] 1.0
v[28,108] 1.0
v[28,109] 1.0
v[28,110] 1.0
v[28,111] 1.0
v[28,112] 1.0
v[28,113] 1.0
v[28,114

In [93]:
df = pd.read_excel("G_30_seed120.xlsx")
NODE_POS = {}
x_list = []
y_list = []
for i in range(len(df)):
    NODE_POS[i+1] = (int(df.iloc[i][0]*2), int(df.iloc[i][1])*2)
    x_list.append(int(df.iloc[i][0]))
    y_list.append(int(df.iloc[i][1]))
print(NODE_POS)
print(type(30))
print(type(x_list[0]))

{1: (514, 188), 2: (952, 634), 3: (588, 940), 4: (90, 792), 5: (222, 250), 6: (320, 738), 7: (940, 706), 8: (316, 128), 9: (286, 216), 10: (502, 508), 11: (114, 208), 12: (878, 700), 13: (94, 866), 14: (448, 384), 15: (804, 688), 16: (436, 382), 17: (544, 834), 18: (708, 86), 19: (28, 356), 20: (288, 778), 21: (606, 138), 22: (420, 576), 23: (480, 258), 24: (688, 280), 25: (994, 378), 26: (502, 802), 27: (350, 900), 28: (552, 540), 29: (236, 330), 30: (526, 818)}
<class 'int'>
<class 'int'>


In [94]:
import json
data={}

print(NODE_POS)
# NODE_POS = {1:(30, 300), 2:(100, 100), 3:(500, 250), 4:(450, 600), 5:(1000, 150)}
print(NODE_POS)
data['NODE_POS'] = NODE_POS
data['N'] = list(N)
data['N_D'] = list(N_D)
data['N_F'] = list(N_F)
data['K'] = list(K)
data['A_p'] = list([str(i) for i in A_p])
data['A_f'] = list([str(i) for i in A_f])
data['T'] = T
data['q'] = Q
data['b'] = b
data['p'] = P
data['h'] = H
# Q={1:6,2:2,3:4,5:6}                  #q=單位時間火在i點燒的燃料量
# b={1:18,2:8,3:12,5:18}               #b=節點i的價值
# P=3                                  #1號消防車單位時間處理的燃料量
# H={1:2,2:3,3:3,5:2}                  #h=火燒i點需要的時間

temp = {}
for (i, j, k, t) in x:
    temp[str((i, j, k, t))] = x[i, j, k, t].X
data['x'] = temp

temp = {}
for (k, t, i) in w:
    temp[str((k, t, i))] = w[k, t, i].X
data['w'] = temp

temp = {}
for (t, i) in u:
    temp[str((t, i))] = u[t, i].X
data['u'] = temp

temp = {}
for (t, i, k) in u_bar:
    temp[str((t, i, k))] = u_bar[t, i, k].X
data['u_bar'] = temp

temp = {}
for (t, i) in v:
    temp[str((t, i))] = v[t, i].X
data['v']  = temp

temp = {}
for (t, i) in v_bar:
    temp[str((t, i))] = v_bar[t, i].X
data['v_bar'] = temp
json_data = json.dumps(data)
with open("data.json", "w") as file:
    file.write(json.dumps(data))

{1: (514, 188), 2: (952, 634), 3: (588, 940), 4: (90, 792), 5: (222, 250), 6: (320, 738), 7: (940, 706), 8: (316, 128), 9: (286, 216), 10: (502, 508), 11: (114, 208), 12: (878, 700), 13: (94, 866), 14: (448, 384), 15: (804, 688), 16: (436, 382), 17: (544, 834), 18: (708, 86), 19: (28, 356), 20: (288, 778), 21: (606, 138), 22: (420, 576), 23: (480, 258), 24: (688, 280), 25: (994, 378), 26: (502, 802), 27: (350, 900), 28: (552, 540), 29: (236, 330), 30: (526, 818)}
{1: (514, 188), 2: (952, 634), 3: (588, 940), 4: (90, 792), 5: (222, 250), 6: (320, 738), 7: (940, 706), 8: (316, 128), 9: (286, 216), 10: (502, 508), 11: (114, 208), 12: (878, 700), 13: (94, 866), 14: (448, 384), 15: (804, 688), 16: (436, 382), 17: (544, 834), 18: (708, 86), 19: (28, 356), 20: (288, 778), 21: (606, 138), 22: (420, 576), 23: (480, 258), 24: (688, 280), 25: (994, 378), 26: (502, 802), 27: (350, 900), 28: (552, 540), 29: (236, 330), 30: (526, 818)}


In [95]:
print(data['A_p'])

['(1, 1)', '(2, 2)', '(3, 3)', '(4, 4)', '(5, 5)', '(6, 6)', '(7, 7)', '(8, 8)', '(9, 9)', '(10, 10)', '(11, 11)', '(12, 12)', '(13, 13)', '(14, 14)', '(15, 15)', '(16, 16)', '(17, 17)', '(18, 18)', '(19, 19)', '(20, 20)', '(21, 21)', '(22, 22)', '(23, 23)', '(24, 24)', '(25, 25)', '(26, 26)', '(27, 27)', '(28, 28)', '(29, 29)', '(30, 30)', '(14, 23)', '(23, 14)', '(5, 8)', '(8, 5)', '(16, 22)', '(22, 16)', '(23, 24)', '(24, 23)', '(1, 21)', '(21, 1)', '(2, 15)', '(15, 2)', '(20, 27)', '(27, 20)', '(2, 25)', '(25, 2)', '(1, 8)', '(8, 1)', '(6, 20)', '(20, 6)', '(3, 7)', '(7, 3)', '(2, 7)', '(7, 2)', '(3, 13)', '(13, 3)', '(12, 15)', '(15, 12)', '(9, 29)', '(29, 9)', '(2, 12)', '(12, 2)', '(8, 23)', '(23, 8)', '(1, 23)', '(23, 1)', '(3, 27)', '(27, 3)', '(8, 21)', '(21, 8)', '(26, 27)', '(27, 26)', '(19, 29)', '(29, 19)', '(22, 28)', '(28, 22)', '(10, 22)', '(22, 10)', '(18, 21)', '(21, 18)', '(5, 9)', '(9, 5)', '(6, 19)', '(19, 6)', '(11, 19)', '(19, 11)', '(15, 28)', '(28, 15)', '(6, 